In [64]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import time
import seaborn as sns
from matplotlib import pyplot as plt

import tensorflow as tf
#from keras.utils import to_categorical
import feather
import random
from datetime import *
import warnings
warnings.filterwarnings("ignore")

In [28]:
FILENAME = "Datasets/final_datasets/merged_data"

In [29]:
data = feather.read_dataframe(f"{FILENAME}.feather")

In [35]:
# Remove those one-hot encoded columns in the dataset created by Abhi

def in_name(c, one_hots):
    for col_name in one_hots:
        if c.startswith(f"{col_name}_"):
            return True
    return False

one_hots = ['Wind_Direction', 'Timezone','State', 'Weather_Condition']
remove = [col for col in data.columns if in_name(col, one_hots)]
df = data.drop(remove, axis=1)


In [36]:
# Remove missing values
df = df.dropna().reset_index(drop=True)
print(len(df))

2922400


In [37]:
df['Severity'] = df['Severity'].astype(int)

# Clean categorical features

### Clean wind direction

* "CALM" = ["Calm", "CALM"]
* "W" = ["W", "West", "WSW", "WNW"] 
* "S" = ["S", "South", "SSW", "SSE"] 
* "N" = ["N", "North", "NNW", "NNE"] 
* "E" = ["E", "East", "ESE", "ENE"]
* "VAR" = ["VAR", "Variable"]

In [38]:
df.loc[df['Wind_Direction']=='Calm','Wind_Direction'] = 'CALM'
df.loc[(df['Wind_Direction']=='West')|(df['Wind_Direction']=='WSW')|(df['Wind_Direction']=='WNW'),'Wind_Direction'] = 'W'
df.loc[(df['Wind_Direction']=='South')|(df['Wind_Direction']=='SSW')|(df['Wind_Direction']=='SSE'),'Wind_Direction'] = 'S'
df.loc[(df['Wind_Direction']=='North')|(df['Wind_Direction']=='NNW')|(df['Wind_Direction']=='NNE'),'Wind_Direction'] = 'N'
df.loc[(df['Wind_Direction']=='East')|(df['Wind_Direction']=='ESE')|(df['Wind_Direction']=='ENE'),'Wind_Direction'] = 'E'
df.loc[df['Wind_Direction']=='Variable','Wind_Direction'] = 'VAR'
print("Wind Direction after simplification: ", df['Wind_Direction'].unique())

Wind Direction after simplification:  ['SW' 'S' 'W' 'NW' 'N' 'VAR' 'SE' 'E' 'NE' 'CALM']


### Clean weather condition

In [ ]:
Counter(df['Weather_Condition'])

https://www.timeanddate.com/weather/glossary.html <br>

**Create features according the different weather conditions** <br>
Scattered clouds, partly cloudy = PARTLY CLOUDY <br>
Overcast, Mostly cloudly, Cloudy = CLOUDY <br>
Fair, Clear = CLEAR <br>
Snow (light snow, heavy snow), and wintry mix  = SNOW <br>
drizzle, light rain/drizzle, light freezing rain/drizzle = LIGHT RAIN <br>
Heavy Rain, heavy thunderstorms, heavy t-storm = HEAVY RAIN <br>
Haze, fog, mist, smoke = fog     *(see https://www.worldatlas.com/articles/what-are-the-differences-between-mist-haze-and-fog.html)*<br>
Rain, light thunderstorms, thunderstorms, t-storm, thunder = RAIN <br>

In [ ]:
def get_condition(string):
    
    conditions = ['Clear', 'Partly Cloudy', 'Cloudy', 'Snow', 'Light Rain', 'Heavy Rain', 'Rain', 'Fog'] 
    strings = [['clear', 'fair'],
              ['partly cloudy', 'scattered clouds'],
              ['overcast', 'mostly cloudy', 'cloudy'],
              ['snow', 'wintry mix'],
              ['drizzle', 'light rain', 'light freezing rain'],
              ['heavy rain', 'heavy thunderstorms', 'heavy t-storm'],
              ['rain', 'thunderstorms', 't-storm', 'thunder', 'showers'],
              ['fog', 'haze', 'mist', 'smoke']]

    for j, k in enumerate(strings):
        for val in k:
            if val in string.lower():
                return conditions[j]
    return 'Others'

In [ ]:
res = []
for i in df['Weather_Condition'].values:
    res.append(get_condition(i))

df['Condition'] = res
df[['Condition', 'Weather_Condition']]

In [ ]:
conditions = ['Clear', 'Partly Cloudy', 'Cloudy', 'Snow', 'Light Rain', 'Heavy Rain', 'Rain', 'Fog', 'Others'] 

for i in conditions:
    print(i, ":")
    print(set(df[df['Condition']==i]['Weather_Condition']))
    print(sum(df['Condition']==i))
    print("\n")
    
df = df[df['Condition']!='Others']

In [ ]:
# One hot encode ['Wind_Direction', 'Timezone', 'State', 'Condition']

one_hots = ['Wind_Direction', 'Timezone','State', 'Condition']
oh = pd.DataFrame()
for c in one_hots:
    dummies = pd.get_dummies(df[c], prefix=c)
    oh = pd.concat([oh, dummies], axis=1)

df = pd.concat([df, oh], axis=1)

In [ ]:
# Save this dataframe
df = df.reset_index(drop = True)
df.to_feather("{}_v1.feather".format(FILENAME))

# SHOW CORRELATION BELOW (CHANGE THIS TITLE)

#### With numeric attributes

In [ ]:
# Filter the data
#BUM NOEXIT ROUNABOUT GIVEWAY removed due to badly distributed data
df = feather.read_dataframe("{}_v1.feather".format(FILENAME))
drop = ['Unnamed: 0', 'ID', 'Start_Lat', 'Start_Lng', 'Start_Time', 'End_Time', 'Weather_Timestamp', 'Description', 
        'Distance(mi)', 
        'County', 'City', 'Airport_Code', 
        'Wind_Direction', 'Timezone', 'State', 'Condition', 'Weather_Condition',
        'Street', 'Bump', 'No_Exit', 'Roundabout', 'Give_Way', 'Traffic_Calming',
        "Wind_Direction"]

df = df.drop(drop, axis=1).reset_index(drop=True)

# Convert to feather file format
df.to_feather("{}_numeric_v2.feather".format(FILENAME))

In [ ]:
del(df)

#### without numeric attributes

In [ ]:
# Filter the data
#BUM NOEXIT ROUNABOUT GIVEWAY removed due to badly distributed data
df = feather.read_dataframe("{}_v1.feather".format(FILENAME))
drop = ['Unnamed: 0', 'ID', 'Start_Lat', 'Start_Lng', 'Start_Time', 'End_Time', 'Weather_Timestamp', 'Description', 
        'Distance(mi)', 
        'County', 'City', 'Airport_Code', 
        'Wind_Direction', 'Timezone', 'State', 'Condition', 'Weather_Condition',
        'Street', 'Bump', 'No_Exit', 'Roundabout', 'Give_Way', 'Traffic_Calming',
       'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)',
       'Population_County', 'Drive_County', 'Transit_County', 'Walk_County', 'MedianHouseholdIncome_County']

df = df.drop(drop, axis=1).reset_index(drop=True)

# Convert to feather file format
df.to_feather("{}_v2.feather".format(FILENAME))

In [ ]:
# Clear memory
del(df)

# Model

In [ ]:
#function for removing outliers in numeric data
def outlier_removal(data,columns):
    for col in columns:
        Q1 = np.quantile(data[col].values,0.25)
        Q3 = np.quantile(data[col].values,0.75)
        IQR = Q3-Q1
        
        lower = Q1 - 1.5*IQR
        upper = Q3+ 1.5*IQR
        
        data = data[(data[col]>=lower) & (data[col]<=upper)]
    return data

# Model Function
def Neural_Network(data,neuron_size,epoch_size):
    scaler=MinMaxScaler(feature_range=(0,1))
    X = scaler.fit_transform(data)
    
    skf = StratifiedKFold(n_splits=3, random_state=30034, shuffle=False)

    fold_no = 1
    accs = []  # store each accuracy


    for train_index, test_index in skf.split(X, labels):

        # One hot encode labels
        print("FOLD NUMBER = ", str(fold_no))


        X_train = X[train_index]
        X_test = X[test_index]

        y_train = labels[train_index]
        y_test = labels[test_index]

        EPOCHS = epoch_size
        BATCH_SIZE = 128


        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(neuron_size,input_dim=X_train.shape[1], activation='relu'))
        model.add(tf.keras.layers.Dense(int(2/3*neuron_size), activation=tf.nn.leaky_relu))
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
        (test_loss, test_acc) = model.evaluate(X_test, y_test)
        print('Accuracy:', test_acc)
        accs.append(test_acc)



        y_preds = model.predict(X_test)
        y_preds = np.where(y_preds < 0.5,0,1)


        print("accuracy:", accuracy_score(y_test,y_preds))

        cm=confusion_matrix(y_test,y_preds)
        cm = pd.DataFrame(cm, index = [i for i in "01"],
                          columns = [i for i in "01"])
        cm.index.name = 'Actual'
        cm.columns.name = 'Predicted'
        sns.heatmap(cm, annot=True, 
                    cmap='Blues', cbar=False)
        plt.show()

        print("============================================")
        fold_no+=1

    return accs

### Model evaluation with numeric

In [ ]:
data['Grouped_Severity'] = np.where(data['Severity']<=2, 0, 1)
data = feather.read_dataframe("{}_numeric_v2.feather".format(FILENAME))
labels = data['Severity']
X = data.drop(['Severity',"Grouped_Severity"], axis=1)

accuracies = Neural_Network(data,50,10)
for i in range(1,4):
    print("Fold ",i," Accuracy: ",accuracies[i])
print("Average Accuracy: ",np.mean(accuracies))

### Model evaluation without numeric

In [ ]:
data = feather.read_dataframe("{}_v2.feather".format(FILENAME))
labels = data['Severity']
X = data.drop(['Severity',"Grouped_Severity"], axis=1)

accuracies = Neural_Network(data,50,10)
for i in range(1,4):
    print("Fold ",i," Accuracy: ",accuracies[i])
print("Average Accuracy: ",np.mean(accuracies))

### Finding best neuron size

In [ ]:
data = feather.read_dataframe("{}_numeric_v2.feather".format(FILENAME))

data['Grouped_Severity'] = np.where(data['Severity']<=2, 0, 1)    
y = data['Grouped_Severity']
X = data.drop(['Severity',"Grouped_Severity"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30034)

max_neurons = X_train.shape[1]
neuron_size = range(max_neurons,4,-10)

accs = []

for i in neuron_size:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(i,input_dim=X_train.shape[1], activation='relu'))
    model.add(tf.keras.layers.Dense(int(2/3*i), activation=tf.nn.leaky_relu))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=10, batch_size=128)
    (test_loss, test_acc) = model.evaluate(X_test, y_test)
    print('Accuracy:', test_acc)
    accs.append(test_acc)

#plotting line graph of accuracy vs neuron_size
sns.set_style("darkgrid")
plt.plot(neuron_size, accuracies, label="Accuracy")

    
best_neuron_size = neuron_size[np.argmax(accs)]

### Performing  outlier removal

In [45]:
data = feather.read_dataframe("{}_numeric_v2.feather".format(FILENAME))

numeric_cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 
        'Wind_Speed(mph)', 'Precipitation(in)',
       'Population_County', 'Drive_County', 'Transit_County', 'Walk_County', 'MedianHouseholdIncome_County']
data = outlier_removal(data,numeric_cols).reset_index(drop = True)

In [ ]:
#Boxplot after outlier removal
cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 
        'Wind_Speed(mph)', 'Precipitation(in)',
       'Population_County', 'Drive_County', 'Transit_County', 'Walk_County', 'MedianHouseholdIncome_County']

sns.set_style('darkgrid')
f, axes = plt.subplots(6,2, figsize=(15,20))
y = 0;
for name in cols[1:]:
    i, j = divmod(y, 2)
    sns.boxplot(x=data[name], ax=axes[i, j])
    y = y+1

plt.tight_layout()
plt.show()

In [65]:
data['Grouped_Severity'] = np.where(data['Severity']<=2, 0, 1)
labels = data['Grouped_Severity']
#dropping percipitation and visibility due to all values removed as outliers
X = data.drop(['Severity',"Grouped_Severity",'Precipitation(in)','Visibility(mi)'], axis=1)

accuracies = Neural_Network(data,best_neuron_size,20)
for i in range(1,4):
    print("Fold ",i," Accuracy: ",accuracies[i])
print("Average Accuracy: ",np.mean(accuracies))

FOLD NUMBER =  1
Epoch 1/20
850661/850661 [==============================] - 8s 9us/sample - loss: 0.4992 - acc: 0.7364
Epoch 2/20
850661/850661 [==============================] - 8s 9us/sample - loss: 0.4837 - acc: 0.7486
Epoch 3/20
850661/850661 [==============================] - 8s 9us/sample - loss: 0.4784 - acc: 0.7526
Epoch 4/20
850661/850661 [==============================] - 8s 9us/sample - loss: 0.4754 - acc: 0.7551
Epoch 5/20
221184/850661 [======>.......................] - ETA: 6s - loss: 0.4722 - acc: 0.7563

KeyboardInterrupt: 